In [20]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle

from pympler import asizeof

from matplotlib import pyplot as plt

In [2]:
#Get all data from ep experiemnts
date_time_str = "2023/09/05/"
save_csv = False
name_cs_str = "CS1"      
meth_name_str_list = ["A1", "B1", "A2", "B2", "C2"]
# study_param_list = ["CONSTANT"]
study_param_list = ["CONSTANT", "DECAY", "BOYLE", "JASRASARIA"]
study_id = "ep"

#Get all ep_data  
df, theta_true = get_study_data_org(date_time_str, name_cs_str, meth_name_str_list, study_id, 
                                    study_param_list, save_csv = False)

In [2]:
#Get all data from sf experiemnts
date_time_str = "2023/09/06/"
# date_time_str = "No_Date/"
save_csv = False
name_cs_str = "CS1" 
meth_name_str_list = ["A1", "B1", "A2", "B2", "C2"]
# meth_name_str_list = ["C2"]
f = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# f = [0.9]
precision = 1
study_param_list = [f"{number:.{precision}f}" for number in f]
study_id = "sf"

#Get all sf_data  
df, theta_true = get_study_data_org(date_time_str, name_cs_str, meth_name_str_list, study_id, 
                                    study_param_list, save_csv = False)

In [21]:
#From signac
import signac
date_time_str = None
meth_name_str_list = [1, 2, 5]
study_id = "ep"
save_csv = False
cs_name_val = 3
project = signac.get_project()
df = pd.DataFrame()
for meth_name_val in meth_name_str_list:
    df_piece, name_cs_str, theta_true = get_study_data_signac(project, cs_name_val, meth_name_val, study_id, save_csv)
    df = pd.concat([df, df_piece], ignore_index=True)

In [22]:
#Get Best Data from ep experiment
save_csv = False
df_best = get_best_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_best

# #Get Mean Data from ep experiment
# df_mean = get_mean_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)
# df_mean

# #Get Median Data from ep experiment
# df_median = get_median_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)

# #Get Median of Best Data from ep experiment
# df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, date_time_str, save_csv)
# df_median_best

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,EP Method Val,BO Method,Max Evals,Total Run Time,L2 Norm Theta
8,4,0,965122.892879,1.0,0.0,"[-265.30631412692253, -127.41457030190627, -15...",-2838295114.154768,27234410.355817,"[-182.02591368957226, -5.853603953180625, -209...",27234410.355817,"[-182.02591368957226, -5.853603953180625, -209...",1.878233,1,A1,2,3.756466,103.865936
38,4,0,13.780011,1.0,0.0,"[-265.30631412692253, -127.41457030190627, -15...",31144.141877,1276589.250046,"[-300.0, 0.0, -150.0, 5.0, -1.4664256803126707...",1276589.250046,"[-300.0, 0.0, -150.0, 5.0, -1.4664256803126707...",2.596007,1,B1,2,5.192015,143.225235
65,2,1,965122.892879,1.0,0.0,"[-128.74303135953963, -27.950809329157465, -19...",2066669.782738,2122989.849739,"[-111.36642801820967, -155.06134161982698, -17...",2122989.849739,"[-111.36642801820967, -155.06134161982698, -17...",147.339926,1,C2,2,294.679852,104.747674


In [23]:
#From signac
import signac
date_time_str = None
meth_name_str_list = [1, 2, 3, 5]
study_id = "sf"
save_csv = False
cs_name_val = 1
project = signac.get_project()
df = pd.DataFrame()
f = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# name_cs_str = "CS1" 
# meth_name_str_list = ["A1", "B1", "A2"]

#Get all sf data signac
df = pd.DataFrame()
for meth_name_val in meth_name_str_list:
    df_piece, name_cs_str, theta_true = get_study_data_signac(project, cs_name_val, meth_name_val, study_id, save_csv)
    df = pd.concat([df, df_piece], ignore_index=True)
    
df = df.sort_values(by=['Sep Fact'], ascending=True)

In [24]:
#Get Best Data from sf experiment
df_best = get_best_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_best

# #Get Median Data from sf experiment
# df_median = get_median_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)

# #Get Median of Best Data from sf experiment
# df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, date_time_str, save_csv)
# df_median_best

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,Sep Fact,BO Method,Max Evals,Total Run Time,L2 Norm Theta
1122,4,8,0.000004,0.92,0.000002,"[1.612228332790134, -0.3873393461459665]",0.0,0.0,"[0.9999142826680114, -1.0000512451832275]",0.0,"[0.9999142826680114, -1.0000512451832275]",0.340214,0.6,A2,9,3.061926,0.000100
236,3,99,0.006709,1413503456553.501465,9483051748.006243,"[1.0492493531520104, -0.9911714417185903]",0.011268,0.002615,"[1.0160563361026016, -0.9989616997723216]",0.002615,"[1.0160563361026016, -0.9989616997723216]",0.42838,0.6,A1,100,42.837977,0.016090
3229,2,32,0.0,85222.692992,0.002344,"[1.0000125789760266, -0.9999981907064244]",0.0,0.0,"[1.0000114521493373, -0.999997851613909]",0.0,"[1.0000114521493373, -0.999997851613909]",1.519454,0.7,C2,33,50.141998,0.000012
888,2,1,2.267023,0.017436,0.0,"[-0.12469795110750015, 1.8446633771315404]",5.561258,0.171544,"[1.0047936613057218, -1.0709834168092685]",0.171544,"[1.0047936613057218, -1.0709834168092685]",0.206683,0.7,B1,2,0.413365,0.071145
